In [1]:
import numpy as np
import os
import copy
import pickle
import heapq
import matplotlib.pyplot as plt
import pandas as pd
from preprocessing_helper import *
import re

In [6]:
cwd = os.getcwd()
parent_wd = cwd.replace('/preprocessing', '')
left_hold_feature_path = parent_wd + '/raw_data/HoldFeature2016LeftHand.csv'
right_hold_feature_path = parent_wd + '/raw_data/HoldFeature2016RightHand.csv'
generated_problem_path = parent_wd + '/out/ProblemOfDeepRouteSet_v1'

In [7]:
raw_data_path = parent_wd + '/preprocessing/processed_data_xy_mode' 
with open(raw_data_path, 'rb') as f:
    MoonBoard_2016_raw = pickle.load(f)  

In [8]:
hold_feature_path = parent_wd + '/raw_data/HoldFeature2016.xlsx'
features = pd.read_excel(hold_feature_path, dtype=str)

# convert features from pd dataframe to dictionary
feature_dict = {}
for index in features.index:
    feature_item = features.loc[index]
    feature_dict[(int(feature_item['X_coord']), int(feature_item['Y_coord']))] = np.array(
        list(feature_item['Difficulties'])).astype(int)

### 1-1. Read hand features

In [5]:
LeftHandfeatures = pd.read_csv(left_hold_feature_path, dtype=str)
RightHandfeatures = pd.read_csv(right_hold_feature_path, dtype=str)

# convert features from pd dataframe to dictionary of left and right hand
RightHandfeature_dict = {}
LeftHandfeature_dict = {}
for index in RightHandfeatures.index:
    LeftHandfeature_item = LeftHandfeatures.loc[index]
    LeftHandfeature_dict[(int(LeftHandfeature_item['X_coord']), int(LeftHandfeature_item['Y_coord']))] = np.array(
        list(LeftHandfeature_item['Difficulties'])).astype(int)
    RightHandfeature_item = RightHandfeatures.loc[index]
    RightHandfeature_dict[(int(RightHandfeature_item['X_coord']), int(RightHandfeature_item['Y_coord']))] = np.array(
        list(RightHandfeature_item['Difficulties'])).astype(int)

### 1-2. Read generated sequence

In [9]:
with open(generated_problem_path, 'rb') as f:
    generated_problems = pickle.load(f)

In [32]:
moveGenerator(generated_problems[0])

AttributeError: 'list' object has no attribute 'handSequence'

In [6]:
def convert_coordinates(coord):
    x_dict = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10}
    x_str, y_str = coord
    x = x_dict[x_str]
    y = int(y_str) - 1
    return (x, y)

In [22]:
def convert_to_pre_beta(gen_seq_list):
    temp = re.compile("([a-zA-Z]+)([0-9]+)(-)([a-zA-Z]+)")
    n_move = len(gen_seq_list)
    gen_seq_split = np.random.rand(n_move, 10)
    for i, move in enumerate(gen_seq_list):
        res = temp.match(move).groups() 
        (x, y) = convert_coordinates((res[0], res[1]))
        gen_seq_split[i, 6:8] = [x, y]
        gen_seq_split[i, 0:6] = feature_dict[(x, y)]
        if i == 0 or i == 1:
            gen_seq_split[i, 8:] = [1, 0]
        elif i == n_move - 1:
            gen_seq_split[i, 8:] = [0, 1]
        else:
            gen_seq_split[i, 8:] = [0, 0]
        
    if np.array_equal(gen_seq_split[0, 6:8], gen_seq_split[1, 6:8]):
        gen_seq_split = np.delete(gen_seq_split, 0, 0)
    return gen_seq_split

def convert_to_data_dict(generated_problems, start_key):
    output = {}
    keynum = start_key
    for gen_seq_list in generated_problems:
        gen_seq_split = convert_to_pre_beta(gen_seq_list)
        key = 'gen' + str(keynum)
        output[key] = gen_seq_split.T
        keynum += 1
    return output

In [23]:
gen_processed_output = convert_to_data_dict(generated_problems, start_key = 0)

In [25]:
save_path = cwd + '/gen_move_seq_X'
output, fail_list = moveGeneratorForAllProblem(gen_processed_output, save_path, print_result = False)
print(fail_list)

result saved.
[]


### Normalization

In [12]:
n_sample = len(generated_problems)
X_seq_data_merge = np.zeros((n_sample, 12, 22))
keys_seq_merge = []
tmax_seq_merge = np.zeros(n_sample)

i = 0
for key, value in generated_problems.items():
    X_data = value.T
    X_seq_data_merge[i, 0:X_data.shape[0], :] = X_data
    keys_seq_merge.append(key)
    tmax_seq_merge[i] = X_data.shape[0]
    i = i + 1

In [13]:
test_set_gen = {'X': X_seq_data_merge,   
                'keys': keys_seq_merge, 
                'tmax': tmax_seq_merge}

In [14]:
test_set_gen_normalized = normalization(test_set_gen)

In [15]:
test_gen_path = parent_wd + '/preprocessing/test_set_gen_v1'

save_pickle(test_set_gen_normalized, test_gen_path)